# Рекомендательные системы на основе SVD алгоритма

**Collaborative Filtering** - подход, основанный на отношениях между пользователями и товаром, при этом нет никакой информации о пользователях или товарах. 

* Пример явного взаимодействия: рейтинг между пользователям и товарами.
* Неявного: клик, просмотр или покупка не очевидны с точки зрения рейтинга.

**User-item matrix** (матрица пользователь-товар) - это таблица, в которой каждая строка представляет собой пользователя, каждый столбец - товар, а каждая ячейка содержит информацию о взаимодействии между пользователем и товаром. Например, в ячейке может быть указано количество раз, которое пользователь купил товар, или оценка, которую пользователь поставил товару. User-item matrix используется в рекомендательных системах для определения предпочтений пользователей и предложения им наиболее подходящих товаров.

Далее, $R$ - user-item matrix.

**Singular Value Decomposition (SVD)** 

Можно применить [матричную факторизацию](https://translated.turbopages.org/proxy_u/en-ru.ru.0832cecd-64b5506e-d08cf623-74722d776562/https/en.wikipedia.org/wiki/Matrix_factorization_(recommender_systems)). Часто, матричная факторизация применяется в области уменьшения размерности, где мы пытаемся уменьшить количество элементов, сохраняя при этом соответсующую информацию. Так дело обстоит с [основным компонентным анализом (PCA)](https://ru.wikipedia.org/wiki/Метод_главных_компонент) и очень похожим [разложение по сингулярному значению (SVD)](https://ru.wikipedia.org/wiki/Сингулярное_разложение).

$$R_{m \times n} = U_{m \times m} \Sigma_{m \times n} V^{T}_{n \times n} $$

где $\Sigma$  — диагональная матрица с неотрицательными сингулярными числами, $U$ и $V$ - ортогональные матрицы и $U^T U = I$.

При этом, матрица $R$ - очень большая и разряженная. 

**Метод главных компонент ([principal component analysis (PSA)](https://ru.wikipedia.org/wiki/Метод_главных_компонент))** — один из основных способов уменьшить размерность данных, потеряв наименьшее количество информации. 
Вычисление главных компонент может быть сведено к вычислению SVD разложения матрицы данных или к вычислению собственных векторов и собственных значений ковариационной матрицы исходных данных. 

Математическое содержание метода главных компонент — это SVD разложение ковариационной матрицы $R$, то есть представление пространства данных в виде суммы взаимно ортогональных собственных подпространств $R$, а самой матрицы $R$ — в виде линейной комбинации ортогональных проекторов на эти подпространства с коэффициентами $\lambda_i$. 

Оставив $d$-первых (главных) компонент ($\lambda_i$) в SVD разложении, получим наиболее точное приближение к $R$ по норме Фробениуса.  
 
$ R_{m \times n} = P_{m \times n} Q_{n \times n} $

$ P = S_{m \times m} V_{m \times n} $ 

$ Q = D^{T}_{n \times n} $

$P$ и $Q$ - взаимно ортогональные собственные подпространства $R$.

Тогда $i$-aя строка $U$ - это представление $i$-го пльзователя, а $j$-ый столбец $V$ - представленте $j$-го товара. Их отношение (рейтинг) выражается с помощью скалярного произведения:

$$R[i][j] = \langle U[i, :], V[:, j] \rangle $$

Матрицы $U$ и $Q$ найдем с помощью градиентного спуска. 

В качестве $Loss$ возьмем $RMSE$:

$$ 
    RMSE = \frac{1}{\lvert D \rvert} \sum_{(i,j) \in D} (\hat{R}_{i, j} - R_{i, j})^2 = 
    \frac{1}{\lvert D \rvert} \sum_{(i, j) \in D} (P_{i} \cdot Q_{j} - R_{i, j})^2
$$

Посчитав градиенты по $P$ и по $Q$, получим формулы обновления весов:

<!-- $\nabla RMSE_p = \frac{d}{dp}RMSE = \frac{1}{\lvert D \rvert} \cdot 2 \cdot \sum_{(u,m) \in D} (\hat{r}_{u,m} - r_{u,m}) \cdot q $

$ \nabla RMSE_q = \frac{d}{dq}RMSE = \frac{1}{\lvert D \rvert} \cdot 2 \cdot \sum_{(u,m) \in D} (\hat{r}_{u,m} - r_{u,m}) \cdot p $ -->

$\begin{cases}
    P[i][k] = P[i][k] - \frac{2}{\lvert D \rvert} \cdot \sum_{(i,j) \in D} (\hat{R}_{i,j} - R_{i,j}) \cdot Q[k][i]
    \\
    Q[k][i] = Q[k][i] - \frac{2}{\lvert D \rvert} \cdot \sum_{(i,j) \in D} (\hat{R}_{i,j} - R_{i,j}) \cdot P[i][k]
\end{cases}
\text{   ,   } k \in {1, d} $

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
k = 10 # максимальная оценка

movies = ['Фантазия', 'ВАЛЛ-И', 'Пиноккио', 'Бемби' , 'Шрэк', 'Дамбо', 'Спасатели', 'Геркулес', 'Кунг-фу Панда', 'Карлсон']
m_movies = len(movies)

users = ['Андрей', 'Аня', 'Алиса', 'Ваня', 'Леша', 'Оксана', 'Саша', 'Паша', 'Сеня', 'Гриша']        
n_users = len(users)

RANDOMSEED = 42
np.random.seed(42)

N = np.random.randint(15, 40)
ind_movies = [np.random.randint(0, m_movies) for _ in range(N)]
ind_users = [np.random.randint(0, n_users) for _ in range(N)]
rating = [np.random.randint(1, k) for _ in range(N)]

In [3]:
ind_movies

[3, 7, 4, 6, 9, 2, 6, 7, 4, 3, 7, 7, 2, 5, 4, 1, 7, 5, 1, 4, 0]

In [4]:
ind_users

[9, 5, 8, 0, 9, 2, 6, 3, 8, 2, 4, 2, 6, 4, 8, 6, 1, 3, 8, 1, 9]

In [5]:
rating

[9, 5, 2, 4, 7, 8, 3, 1, 4, 2, 8, 4, 2, 6, 6, 4, 6, 2, 2, 4, 8]

In [6]:
def get_user_item_matrix(n_users, m_movies, ind_users, ind_movies, rating):
    R = [[0] * m_movies for _ in range(n_users)]
    N = len(ind_users)
    for i in range(N):
        R[ind_users[i]][ind_movies[i]] = rating[i]
    R = np.array(R)
    return R

R = get_user_item_matrix(n_users, m_movies, ind_users, ind_movies, rating)

pd.DataFrame(R, movies, users)

,Андрей,Аня,Алиса,Ваня,Леша,Оксана,Саша,Паша,Сеня,Гриша
Фантазия,0,0,0,0,0,0,4,0,0,0
ВАЛЛ-И,0,0,0,0,4,0,0,6,0,0
Пиноккио,0,0,8,2,0,0,0,4,0,0
Бемби,0,0,0,0,0,2,0,1,0,0
Шрэк,0,0,0,0,0,6,0,8,0,0
Дамбо,0,0,0,0,0,0,0,5,0,0
Спасатели,0,4,2,0,0,0,3,0,0,0
Геркулес,0,0,0,0,0,0,0,0,0,0
Кунг-фу Панда,0,2,0,0,6,0,0,0,0,0
Карлсон,8,0,0,9,0,0,0,0,0,7


$$L = \frac{1}{N} \cdot \sum(r - \het r)^2$$

In [7]:
def MSE(R, U, V):
    mse = 0
    for ind in range(N):
        i = ind_users[ind]
        j = ind_movies[ind]
        mse += ( R[i][j] - np.dot(U[i,:], V[:,j]) )**2 / N
    return mse

In [8]:
def SVD(ind_users, ind_movies, R, d, step, n_iters):
    
    # инициализация матриц разложения
    U = np.random.rand(R.shape[0], d)
    V = np.random.rand(d, R.shape[1])

    start_mse = MSE(R, U, V)
    
    for n in range(n_iters):
        ind = np.random.randint(0, N)
        i = ind_users[ind]
        j = ind_movies[ind]
        
        for k in range(0, d):
            U[i, k] = U[i, k] + step * (R[i][j] - np.dot(U[i, :], V[:, j])) * V[k, j] 
            V[k, j] = V[k, j] + step * (R[i][j] - np.dot(U[i, :], V[:, j])) * U[i, k] 

        mse = MSE(R, U, V)
    return U, V, start_mse, mse

In [9]:
U, V, start_mse, mse = SVD(ind_users, ind_movies, R, 3, 0.1, 3000) 

In [10]:
U

array([[ 1.14925335,  0.88815964,  1.28628278],
       [ 0.57882653,  1.94198177,  0.69771869],
       [ 2.53474404,  0.56804522, -0.93485506],
       [ 0.3704904 , -0.16867209,  1.05388538],
       [ 1.55206751,  2.16957811,  0.85723033],
       [ 1.33365791,  1.18894038,  0.42200087],
       [-0.97242016,  0.72281182,  2.01041876],
       [ 0.92630088,  0.65107703,  0.91495968],
       [ 2.15831628,  0.66739268,  0.06101079],
       [ 3.4193753 ,  1.93082125,  2.14906612]])

In [11]:
V

array([[ 1.13902333,  0.49589526,  3.13577084,  1.51022004,  2.43299874,
         1.61478736,  0.90118881,  1.35917061,  0.25606832,  0.58638664],
       [ 1.20469557,  1.22963668,  2.65392918, -0.11313503,  1.04946214,
         1.02028505,  1.13741072,  2.37850328,  0.13933145,  1.34705423],
       [ 0.82791748,  1.78740018,  1.55738422,  1.88647772,  0.7935608 ,
         1.49336078,  1.5191865 ,  0.85172746,  0.10549426,  1.11397325]])

In [12]:
R[1][2]

0

In [13]:
np.dot(U[1,:], V[:,2])

8.055565517055356

In [14]:
start_mse, mse

(21.37688033760675, 4.97897320664505e-09)

In [16]:
cap_R = np.zeros((R.shape[0], R.shape[1]))

for i in range(n_users):
    for j in range(m_movies):
        cap_R[i][j] = round(np.dot(U[i,:], V[:,j]), 2)
cap_R

array([[ 3.440e+00,  3.960e+00,  7.960e+00,  4.060e+00,  4.750e+00,
         4.680e+00,  4.000e+00,  4.770e+00,  5.500e-01,  3.300e+00],
       [ 3.580e+00,  3.920e+00,  8.060e+00,  1.970e+00,  4.000e+00,
         3.960e+00,  3.790e+00,  6.000e+00,  4.900e-01,  3.730e+00],
       [ 2.800e+00,  2.800e-01,  8.000e+00,  2.000e+00,  6.020e+00,
         3.280e+00,  1.510e+00,  4.000e+00,  6.300e-01,  1.210e+00],
       [ 1.090e+00,  1.860e+00,  2.360e+00,  2.570e+00,  1.560e+00,
         2.000e+00,  1.740e+00,  1.000e+00,  1.800e-01,  1.160e+00],
       [ 5.090e+00,  4.970e+00,  1.196e+01,  3.720e+00,  6.730e+00,
         6.000e+00,  5.170e+00,  8.000e+00,  7.900e-01,  4.790e+00],
       [ 3.300e+00,  2.880e+00,  7.990e+00,  2.680e+00,  4.830e+00,
         4.000e+00,  3.200e+00,  5.000e+00,  5.500e-01,  2.850e+00],
       [ 1.430e+00,  4.000e+00,  2.000e+00,  2.240e+00, -1.000e-02,
         2.170e+00,  3.000e+00,  2.110e+00,  6.000e-02,  2.640e+00],
       [ 2.600e+00,  2.900e+00,  6.060e+0

In [17]:
pd.DataFrame(cap_R, movies, users)

,Андрей,Аня,Алиса,Ваня,Леша,Оксана,Саша,Паша,Сеня,Гриша
Фантазия,3.44,3.96,7.96,4.06,4.75,4.68,4.00,4.77,0.55,3.30
ВАЛЛ-И,3.58,3.92,8.06,1.97,4.00,3.96,3.79,6.00,0.49,3.73
Пиноккио,2.80,0.28,8.00,2.00,6.02,3.28,1.51,4.00,0.63,1.21
Бемби,1.09,1.86,2.36,2.57,1.56,2.00,1.74,1.00,0.18,1.16
Шрэк,5.09,4.97,11.96,3.72,6.73,6.00,5.17,8.00,0.79,4.79
Дамбо,3.30,2.88,7.99,2.68,4.83,4.00,3.20,5.00,0.55,2.85
Спасатели,1.43,4.00,2.00,2.24,-0.01,2.17,3.00,2.11,0.06,2.64
Геркулес,2.60,2.90,6.06,3.05,3.66,3.53,2.97,3.59,0.42,2.44
Кунг-фу Панда,3.31,2.00,8.63,3.30,6.00,4.26,2.80,4.57,0.65,2.23
Карлсон,8.00,7.91,19.19,9.00,12.05,10.70,8.54,11.07,1.37,7.00
